# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

# Импорт

In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


# Предобработка Датасета

## Загружаем DataSet

In [29]:
df = pd.read_csv('moldova_cars_task.csv')
df


,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro)
0,Toyota,Prius,2011.0,Hatchback,195000.0,1800.0,Hybrid,Automatic,7750.0
1,Renault,Grand Scenic,2014.0,Universal,135000.0,1500.0,Diesel,Manual,8550.0
2,Volkswagen,Golf,1998.0,Hatchback,1.0,1400.0,Petrol,Manual,2200.0
3,Renault,Laguna,2012.0,Universal,110000.0,1500.0,Diesel,Manual,6550.0
4,Opel,Astra,2006.0,Universal,200000.0,1600.0,Metan/Propan,Manual,4100.0
...,...,...,...,...,...,...,...,...,...
41002,Dacia,NaN,2015.0,Universal,89000.0,1500.0,Diesel,Manual,7000.0
41003,Renault,Modus,2009.0,Hatchback,225.0,1500.0,Diesel,Manual,4500.0
41004,Mercedes,E Class,2016.0,Sedan,50000.0,1950.0,Diesel,Automatic,29500.0
41005,Mazda,6,2006.0,Combi,370000.0,2000.0,Diesel,Manual,4000.0


In [30]:
df.dtypes

Make                     object
Model                    object
Year                    float64
Style                    object
Distance                float64
Engine_capacity(cm3)    float64
Fuel_type                object
Transmission             object
Price(euro)             float64
dtype: object

## Нивелирование анамалий
- Заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму)
- Удаление дубликатов

In [31]:
df = df.drop_duplicates()
df = df.apply(lambda a: a.fillna(a.mean() if a.dtypes ==
              np.float64 else a.median() if a.dtypes == np.int64 else a.mode()[0]))
df


,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro)
0,Toyota,Prius,2011.0,Hatchback,195000.0,1800.0,Hybrid,Automatic,7750.0
1,Renault,Grand Scenic,2014.0,Universal,135000.0,1500.0,Diesel,Manual,8550.0
2,Volkswagen,Golf,1998.0,Hatchback,1.0,1400.0,Petrol,Manual,2200.0
3,Renault,Laguna,2012.0,Universal,110000.0,1500.0,Diesel,Manual,6550.0
4,Opel,Astra,2006.0,Universal,200000.0,1600.0,Metan/Propan,Manual,4100.0
...,...,...,...,...,...,...,...,...,...
41001,Land Rover,Freelander,2002.0,Crossover,225000.0,1800.0,Metan/Propan,Manual,4400.0
41002,Dacia,E Class,2015.0,Universal,89000.0,1500.0,Diesel,Manual,7000.0
41003,Renault,Modus,2009.0,Hatchback,225.0,1500.0,Diesel,Manual,4500.0
41005,Mazda,6,2006.0,Combi,370000.0,2000.0,Diesel,Manual,4000.0


In [32]:
df.isnull().sum()


Make                    0
Model                   0
Year                    0
Style                   0
Distance                0
Engine_capacity(cm3)    0
Fuel_type               0
Transmission            0
Price(euro)             0
dtype: int64

## Преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума)

In [35]:
def replacer(a):
    x = a.unique()
    return a.map(lambda k: np.where(x == k)[0][0]) if a.dtypes == 'object' else a


In [53]:
names = dict(df.loc[:, list(filter(lambda a: df[a].dtypes == 'object', df.columns))].apply(lambda a: a.unique()))
names

{'Make': array(['Toyota', 'Renault', 'Volkswagen', 'Opel', 'Mercedes', 'BMW',
        'KIA', 'Volvo', 'Nissan', 'Hyundai', 'Audi', 'GAZ', 'Tesla',
        'Lincoln', 'Lexus', 'Dodge', 'Porsche', 'Mazda', 'Dacia',
        'Peugeot', 'Vaz', 'Ford', 'Seat', 'Honda', 'Skoda', 'Chevrolet',
        'Subaru', 'Citroen', 'Fiat', 'Jaguar', 'Infiniti', 'Mitsubishi',
        'Land Rover', 'Suzuki', 'Daewoo', 'Datsun', 'Mini', 'Changan',
        'Rover', 'Saab', 'Cadillac', 'GMC', 'Chrysler', 'Jeep', 'Acura',
        'Smart', 'Isuzu', 'Ravon', 'Moskvich / Izh', 'Chery', 'Hummer',
        'Alfa Romeo', 'UAZ', 'Lancia', 'Zaz', 'Daihatsu', 'Ssangyong',
        'Byd', 'Alta marca', 'LuAZ', 'Maserati', 'Bentley', 'Jac', 'Lifan',
        'Brilliance', 'Pontiac', 'Great Wall', 'Buick', 'Groz', 'BAIC',
        'Ferrari', 'Lada', 'Scion', 'Haval', 'Abarth', 'Haima', 'Zotye',
        'Geely', 'Tata', 'Faw', 'Lamborghini', 'McLaren',
        'Mercedes-Maybach', 'Saturn', 'Aston Martin', 'ARO', 'Xpeng'],
    

In [34]:
df = df.apply(replacer)
df


,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro)
0,0,0,2011.0,0,195000.0,1800.0,0,0,7750.0
1,1,1,2014.0,1,135000.0,1500.0,1,1,8550.0
2,2,2,1998.0,0,1.0,1400.0,2,1,2200.0
3,1,3,2012.0,1,110000.0,1500.0,1,1,6550.0
4,3,4,2006.0,1,200000.0,1600.0,3,1,4100.0
...,...,...,...,...,...,...,...,...,...
41001,32,87,2002.0,6,225000.0,1800.0,3,1,4400.0
41002,18,10,2015.0,1,89000.0,1500.0,1,1,7000.0
41003,1,379,2009.0,0,225.0,1500.0,1,1,4500.0
41005,17,143,2006.0,9,370000.0,2000.0,1,1,4000.0


## Сохранение

In [54]:
df.to_csv('moded__moldova_cars_task.csv', sep='\t')

# Основное задание

## Загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку.

### Загрузите датасет для регрессии.

In [55]:
df = pd.read_csv('moded__moldova_cars_task.csv')

### Выделите целевой признак и предикторы.